In [1]:
%matplotlib inline

from time import time
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import os
from os import path 
from time import time

In [131]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "TRAS.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
from django.core.exceptions import ObjectDoesNotExist
django.setup()
from django.db import transaction
from django.conf import settings

from django_pandas.io import read_frame

from proto.models import Attraction, Review

from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from asgiref.sync import sync_to_async
import nltk

In [206]:
# 디장고 all 리뷰 데이터 불러오고. bulk load?
def preprocess(Reviews_df):
    result_list = [[Review.title,tokenize.sent_tokenize(Review.text)] for Review in Reviews_df]
    return result_list

def sentiment_analysis(Review_list):
    result_list = [[Review[0], sentiment(Review[1])] for Review in Review_list]
    return result_list

def sentiment(review)
    sid = SentimentIntensityAnalyzer()
    
    sid.polarity_scores(Review_sentences[1])['compound']
    result_list = []
    for sentence in sentences_list:
        # print(sentence) 샘플만 작게 해서 테스트
        ss = 
        
        '''
        for k in sorted(ss):
            print('{0}: {1}, '.format(k, ss[k]), end='')
        print()
        '''
        #compound: 0.8316, neg: 0.0, neu: 0.254, pos: 0.746,
    return pd.DataFrame(result_list, columns=['sentence','sentiment'])


def sentences_extraction(sent_df, sample_size):
    sent_df.sort_values(by=['sentiment'],inplace=True)
    
    positive_sample = sent_df[-int(sample_size):][::-1]
    negative_sample = sent_df[:int(sample_size)]
    
    return positive_sample, negative_sample

#star기반 수축 감정 사전 구축

In [76]:
Attractions_query = Attraction.objects.all()      
Attractions_df = read_frame(Attractions_query)

In [81]:
Attractions_name = Attractions_df.name.tolist()

In [73]:
query = Review.objects.all()
All_Reviews_df = read_frame(query)

In [199]:
All_Reviews_df.head()

,id,attraction,star,title,text,date
0,1,Gyeongbokgung Palace,5,"Fantastic area, don t miss",I had the chance to visit the site after a bus...,2019-12-01
1,2,Gyeongbokgung Palace,4,Pretend you're in a period K-drama,Rent a hanbok and soak in the ancient Joseon v...,2019-10-01
2,3,Gyeongbokgung Palace,4,Love to See and Feel the History,Walking around the palace is a unique experien...,2020-02-01
3,4,Gyeongbokgung Palace,5,What an experience,We had a layover in South Korea and decided to...,2020-02-01
4,5,Gyeongbokgung Palace,5,good to visit,you can see the blue house in this area and th...,2020-02-01


In [201]:
sentence_list = preprocess(All_Reviews_df[::10])

In [202]:
sentsent_df = sentiment_analysis(sentence_list)
sentsent_df.head()

,sentence,sentiment
0,I had the chance to visit the site after a bus...,0.6801
1,We visited during a layover at the Incheon Ai...,0.0000
2,If you ever have a long lay-off and land befo...,0.0000
3,I could not stand the airport for 8 hours,0.0000
4,I researched about how this was the principle...,0.0000


In [204]:
pos_sample, neg_sample = sentences_extraction(sentsent_list, sample_size = 10)
print("neg_sample")
for one_neg_sample in neg_sample.values:
    print(one_neg_sample)

neg_sample
['It is difficult to be affected by the torture cell displays when you are surrounded by screaming kids whose teachers do not control them.'
 -0.8625]
['As others have mentioned, if you are unfortunate enough to coincide your visit with a school tour then this really detracts from the experience.'
 -0.4588]
['Avoid using it during rush hour.' -0.296]
['Fun.Well thought out museum that gives an insight into Korea’s history and their relationship to Japan.'
 0.0]
['The museum staff also do not intervene.' 0.0]
['I also liberated a young man who had been locked in a cell by his ‘friends’.This district is lined with international cages and retail stores.'
 0.0]
["But, there's nothing fantastic.- The entrance from the east seems to be main gate with a parking lot."
 0.0]
['Many family visitors.' 0.0]
['People who live in Seoul come and rest here.' 0.0]
['You can spend around 30 mins to 1 hour here.' 0.0]


In [ ]:
# 아래는 리뷰가 다 사라졌길래 다시 bulk create한 내용

In [54]:
def review_date(date_string):
    dict = {
        ' January ':'-01-01',
        ' February ':'-02-01',
        ' March ':'-03-01',
        ' April ':'-04-01',
        ' May ':'-05-01',
        ' June ':'-06-01',
        ' July ':'-07-01',
        ' August ':'-08-01',
        ' September ':'-09-01',
        ' October ':'-10-01',
        ' November ':'-11-01',
        ' December ':'-12-01',
    }
    
    return date_string[-4:] + dict[date_string[:-4]]

In [66]:
review_path = 'C:/Users/taehee/Documents/GitHub/tripReviewAnalysisSystem/크롤러-전처리/원시자료/'
Attraction_list = Attraction.objects.all()
Reviews_list = []
for the_Attraction in Attraction_list:
    Attraction_Reviews_df = pd.read_csv(review_path+ the_Attraction.name+".csv", engine='python', index_col = 0, encoding = 'utf-8')
    for Review_list in Attraction_Reviews_df.values[2:]:
        Reviews_list.append(Review(attraction = the_Attraction,
                       star = Review_list[0],
                      title = Review_list[1],
                      text = Review_list[2],
                      date = review_date(Review_list[3])))

In [69]:
len(Reviews_list)

18534

In [72]:
for num in range(19):
    start = (num-1)*1000
    if start <0:
        start = 0
    end = num*1000
    if end > 18534:
        end = 18534
    Review.objects.bulk_create(Reviews_list[start:end])

In [61]:
max_len = 0
max_text = ""
for Review in Reviews_list:
    if len(Review.text) > max_len:
        max_text = Review.text
        max_len = len(Review.text)

In [62]:
print(max_len)

9136


In [63]:
print(max_text)

The best military museum I’ve ever seen.  Korea’s War Memorial has about 650,000 square-feet under roof, and more than 60 fixed-wing aircraft, choppers, tanks, artillery pieces, missiles, and naval vessels outside in the rain.  You get a crash course in the history of Korean warfare from neolithic spear-fights to modern times:  civil wars between ancient ethnic groups, invasions by the Chinese, Japanese, Mongols, Manchus and so on; the centuries-long Japanese pirate fleet threat; the 19th century attacks by various Western powers; all the way up to the Vietnam War, when more than a third of a million ROK troops supported Saigon.Most of these wars, battles, sieges, victories and defeats will be terra incognita to foreign visitors.  But it’s all interesting and well-presented.  Curatorship is outstanding.   21st century technology helps: if you see locals watching any animation on pirate fleets, city-wall sieges, et cetera, stop and watch the spectacle with them.Most military museums foc